In [33]:
import sqlite3
conn = sqlite3.connect("AirlineDb3.db")
c = conn.cursor()

In [3]:
c.execute('''CREATE TABLE IF NOT EXISTS Flights (flno INTEGER PRIMARY KEY, from_city TEXT, to_city TEXT, distance INTEGER, departs TIME, arrives TIME, price INTEGER)''')
flights_data = [
    (101, 'Bengaluru', 'Frankfurt', 8000, '10:00', '14:00', 50000),
    (102, 'Bengaluru', 'London', 7000, '12:00', '16:00', 60000),
    (103, 'Bengaluru', 'New York', 15000, '08:00', '22:00', 70000),
    (104, 'Delhi', 'Frankfurt', 6500, '11:00', '15:00', 40000),
    (105, 'Mumbai', 'Paris', 8000, '09:00', '13:00', 55000)
]
c.executemany("INSERT OR IGNORE INTO Flights VALUES (?, ?, ?, ?, ?, ?, ?)", flights_data)

print("FLIGHTS:")
for row in c.execute("SELECT * FROM Flights"):
    print(row)

FLIGHTS:
(101, 'Bengaluru', 'Frankfurt', 8000, '10:00', '14:00', 50000)
(102, 'Bengaluru', 'London', 7000, '12:00', '16:00', 60000)
(103, 'Bengaluru', 'New York', 15000, '08:00', '22:00', 70000)
(104, 'Delhi', 'Frankfurt', 6500, '11:00', '15:00', 40000)
(105, 'Mumbai', 'Paris', 8000, '09:00', '13:00', 55000)


In [5]:
c.execute('''CREATE TABLE IF NOT EXISTS Aircraft (aid INTEGER PRIMARY KEY, aname TEXT, cruisingrange INTEGER)''')

aircraft_data = [
    (1, 'Boeing 747', 13000),
    (2, 'Airbus A320', 6000),
    (3, 'Boeing 777', 15000),
    (4, 'Airbus A380', 14000),
    (5, 'Boeing 737', 5000)
]
c.executemany("INSERT OR IGNORE INTO Aircraft VALUES (?, ?, ?)", aircraft_data)

print("\nAIRCRAFT:")
for row in c.execute("SELECT * FROM Aircraft"):
    print(row)


AIRCRAFT:
(1, 'Boeing 747', 13000)
(2, 'Airbus A320', 6000)
(3, 'Boeing 777', 15000)
(4, 'Airbus A380', 14000)
(5, 'Boeing 737', 5000)


In [6]:
c.execute('''CREATE TABLE IF NOT EXISTS Employees (
                eid INTEGER PRIMARY KEY,
                ename TEXT,
                salary INTEGER)''')

employees_data = [
    (1, 'Jerry', 90000),
    (2, 'Alice', 85000),
    (3, 'Tom', 95000),
    (4, 'Spikes', 75000),
    (5, 'Bob', 60000),
    (6, 'John', 40000)
]
c.executemany("INSERT OR IGNORE INTO Employees VALUES (?, ?, ?)", employees_data)

print("\nEMPLOYEES:")
for row in c.execute("SELECT * FROM Employees"):
    print(row)


EMPLOYEES:
(1, 'Jerry', 90000)
(2, 'Alice', 85000)
(3, 'Tom', 95000)
(4, 'Spikes', 75000)
(5, 'Bob', 60000)
(6, 'John', 40000)


In [27]:
c.execute('''CREATE TABLE IF NOT EXISTS Certified (eid INTEGER, aid INTEGER, FOREIGN KEY (eid) REFERENCES Employees(eid), FOREIGN KEY (aid) REFERENCES Aircraft(aid))''')

certified_data = [
    (1, 1),
    (1, 3),
    (1, 4),
    (2, 2),
    (2, 4),
    (2, 5),
    (3, 3),
    (3, 4),
    (4, 1),
    (4, 4)
]
c.executemany("replace INTO Certified VALUES (?, ?)", certified_data)

print("\nCERTIFIED:")
for row in c.execute("SELECT * FROM Certified"):
    print(row)


CERTIFIED:
(1, 1)
(1, 3)
(1, 4)
(2, 2)
(2, 4)
(2, 5)
(3, 3)
(3, 4)
(4, 1)
(4, 4)


In [26]:
c.execute('Drop table Certified')

In [42]:
c.execute('''
    SELECT DISTINCT A.aname
    FROM Aircraft A
    WHERE NOT EXISTS (
        SELECT C.eid
        FROM Certified C
        JOIN Employees E ON C.eid = E.eid
        WHERE C.aid = A.aid AND E.salary <= 80000
    );
''')
result_1 = c.fetchall()
print("\nQuery 1: Aircraft where all certified pilots have salaries > 80,000")
for i in result_1:
    print(i)


Query 1: Aircraft where all certified pilots have salaries > 80,000
('Airbus A320',)
('Boeing 777',)
('Boeing 737',)


In [43]:
c.execute('''
    SELECT DISTINCT A.aname
    FROM AIRCRAFT A
    JOIN CERTIFIED C ON A.aid = C.aid
    JOIN EMPLOYEES E ON C.eid = E.eid
    GROUP BY A.aname
    HAVING MIN(E.salary) > 80000;
''')
result_1 = c.fetchall()
print("\nQuery 1: Aircraft where all certified pilots have salaries > 80,000")
for i in result_1:
    print(i)


Query 1: Aircraft where all certified pilots have salaries > 80,000
('Airbus A320',)
('Boeing 737',)
('Boeing 777',)


In [34]:
# ii. Pilots certified for more than 3 aircraft, find eid and max cruising range
c.execute('''
    SELECT C.eid, MAX(A.cruisingrange)
    FROM Certified C
    JOIN Aircraft A ON C.aid = A.aid
    GROUP BY C.eid;
''')
result_2 = c.fetchall()
print("\nQuery 2: Pilots certified for >3 aircrafts with max cruising range")
for i in result_2:
    print(i)


Query 2: Pilots certified for >3 aircrafts with max cruising range
(1, 15000)
(2, 14000)
(3, 15000)
(4, 14000)


In [49]:

c.execute('''
    SELECT E.ename
    FROM Employees E
    WHERE E.salary < (
          SELECT MIN(F.price)
          FROM Flights F
          WHERE F.from_city = 'Bengaluru' AND F.to_city = 'Frankfurt')
        AND E.eid IN (
          SELECT C.eid
          FROM Certified C
);
''')
result_3 = c.fetchall()
print("\nQuery 3: Pilots with salary < price of cheapest Bengaluru-Frankfurt route")
print(result_3)


Query 3: Pilots with salary < price of cheapest Bengaluru-Frankfurt route
[]


In [30]:
# iv. Aircraft with cruising range > 1000km and average salary of pilots certified
c.execute('''
    SELECT A.aname, AVG(E.salary)
    FROM Aircraft A
    JOIN Certified C ON A.aid = C.aid
    JOIN Employees E ON C.eid = E.eid
    WHERE A.cruisingrange > 1000
    GROUP BY A.aname;
''')
result_4 = c.fetchall()
print("\nQuery 4: Aircraft with cruising range > 1000km and average pilot salary")
print(result_4)


Query 4: Aircraft with cruising range > 1000km and average pilot salary
[('Airbus A320', 85000.0), ('Airbus A380', 86250.0), ('Boeing 737', 85000.0), ('Boeing 747', 82500.0), ('Boeing 777', 92500.0)]


In [31]:
# v. Names of pilots certified for Boeing aircraft
c.execute('''
    SELECT DISTINCT E.ename
    FROM Employees E
    JOIN Certified C ON E.eid = C.eid
    JOIN Aircraft A ON C.aid = A.aid
    WHERE A.aname LIKE 'Boeing%';
''')
result_5 = c.fetchall()
print("\nQuery 5: Pilots certified for Boeing aircraft")
print(result_5)

# Close the connection
conn.commit()
conn.close()


Query 5: Pilots certified for Boeing aircraft
[('Jerry',), ('Alice',), ('Tom',), ('Spikes',)]
